In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
customer = pd.read_csv(r'CUSTOMERS.csv')
geo_location = pd.read_csv(r'GEO_LOCATION.csv')
order_items = pd.read_csv(r'ORDER_ITEMS.csv')
order_payments = pd.read_csv(r'ORDER_PAYMENTS.csv')
order_review_ratings = pd.read_csv(r'ORDER_REVIEW_RATINGS.csv')
orders = pd.read_csv(r'ORDERS.csv')
products = pd.read_csv(r'PRODUCTS.csv')
sellers = pd.read_csv(r'SELLERS.csv')

# 1. Perform Detailed exploratory analysis

In [ ]:
cust_order = pd.merge(customer,orders, how= 'left', left_on = 'customer_id', right_on= 'customer_id')
cust_order_pay = pd.merge(cust_order,order_payments, how= 'left', left_on = 'order_id', right_on= 'order_id')

In [ ]:
def missing_imp(x):
    if x.dtype == 'object':
        x = x.fillna(x.mode()[0])
    elif (x.dtype == 'float64') | (x.dtype == 'int64'):
        x = x.fillna(x.mean())
    else:
        x
    return(x)

cust_order_pay = cust_order_pay.apply(missing_imp)

In [ ]:
cust_order_pay.order_purchase_timestamp = pd.to_datetime(cust_order_pay.order_purchase_timestamp)
cust_order_pay.order_approved_at = pd.to_datetime(cust_order_pay.order_approved_at)
cust_order_pay.order_delivered_carrier_date = pd.to_datetime(cust_order_pay.order_delivered_carrier_date)
cust_order_pay.order_delivered_customer_date = pd.to_datetime(cust_order_pay.order_delivered_customer_date)
cust_order_pay.order_estimated_delivery_date = pd.to_datetime(cust_order_pay.order_estimated_delivery_date)

## a. Define & calculate high level metrics like (Total Revenue, Total quantity, Total products, Total categories, Total sellers, Total locations, Total channels, Total payment methods etc…)

In [ ]:
print('Total Revenue is',cust_order_pay['payment_value'].sum())

Total Revenue is 16009026.220380418


In [ ]:
print('Total quantity is',order_items.order_item_id.sum())

Total quantity is 134936


In [ ]:
print('Total products are',order_items.product_id.nunique())

Total products are 32951


In [ ]:
print('Total categories are',products.product_category_name.nunique())

Total categories are 71


In [ ]:
print('Total sellers are',sellers.seller_id.nunique())

Total sellers are 3095


In [ ]:
print('Total locations are',geo_location.geolocation_zip_code_prefix.nunique())

Total locations are 19015


In [ ]:
print('Total channels are',order_payments.payment_type.unique())

Total channels are ['credit_card' 'UPI' 'voucher' 'debit_card' 'not_defined']


In [ ]:
print('Total payment methods are',order_payments.payment_type.nunique())

Total payment methods are 5


## b. Understanding how many new customers acquired every month

In [ ]:
cust_order_pay['order_purchase_month'] = cust_order_pay.order_purchase_timestamp.dt.month_name()

In [ ]:
cust_total = cust_order_pay.sort_values(by = 'order_purchase_month')

In [ ]:
cust_new = cust_order_pay[['order_purchase_month','customer_unique_id']].sort_values(by = 'order_purchase_month')
cust_new.drop_duplicates(subset= 'customer_unique_id', keep= 'first', inplace= True)

In [ ]:
cust_total_new = pd.merge(cust_total, cust_new, how= 'left', left_on= ['customer_unique_id','order_purchase_month'], right_on= ['customer_unique_id','order_purchase_month'], indicator= True)

In [ ]:
m = cust_total_new._merge
cust_total_new['flag_new'] = np.where(m == 'both',1,0)
cust_total_new.drop(columns= '_merge', inplace= True)

In [ ]:
new_cust = pd.crosstab(cust_total_new.order_purchase_month,cust_total_new.flag_new)
new_cust.rename(columns={0 : 'existing', 1: 'new'}, inplace= True)
new_cust

flag_new,existing,new
order_purchase_month,,
April,0,9780
August,39,11209
December,51,5845
February,99,8739
January,138,8275
July,222,10602
June,238,9617
March,267,10082
May,371,10708


In [ ]:
cust_pct = new_cust.T/new_cust.T.sum()
cust_pct.T

flag_new,existing,new
order_purchase_month,,
April,0.000000,1.000000
August,0.003467,0.996533
December,0.008650,0.991350
February,0.011202,0.988798
January,0.016403,0.983597
July,0.020510,0.979490
June,0.024150,0.975850
March,0.025800,0.974200
May,0.033487,0.966513


## c. Understand the retention of customers on month on month basis

In [ ]:
print('Retention of customers on monthly basis in numbers :')
new_cust.existing

Retention of customers on monthly basis in numbers :


order_purchase_month
April          0
August        39
December      51
February      99
January      138
July         222
June         238
March        267
May          371
November     245
October      208
September    235
Name: existing, dtype: int64

In [ ]:
print('Retention of customers on monthly basis in percentage :')
cust_pct.T.existing

Retention of customers on monthly basis in percentage :


order_purchase_month
April        0.000000
August       0.003467
December     0.008650
February     0.011202
January      0.016403
July         0.020510
June         0.024150
March        0.025800
May          0.033487
November     0.031159
October      0.039954
September    0.051808
Name: existing, dtype: float64

## d. How the revenues from existing/new customers on month on month basis

In [ ]:
monthly_revenue = pd.pivot_table(cust_total_new, index= 'order_purchase_month', columns= 'flag_new', values= 'payment_value', aggfunc= 'sum')

In [ ]:
monthly_revenue.rename(columns={0 : 'existing', 1: 'new'}, inplace= True)
monthly_revenue.fillna(0)

flag_new,existing,new
order_purchase_month,,
April,0.00,1.578574e+06
August,5626.24,1.691195e+06
December,5884.02,8.725371e+05
February,14553.61,1.269818e+06
January,18527.93,1.234964e+06
July,34705.78,1.624218e+06
June,34450.33,1.500707e+06
March,34850.58,1.574665e+06
May,51624.50,1.695276e+06


## e. Understand the trends/seasonality of sales, quantity by category, location, month, week, day, time, channel, payment method etc…

In [ ]:
pdt_item = pd.merge(order_items,products,how= 'left',left_on= 'product_id', right_on= 'product_id')

In [ ]:
cust_order_summ = pd.merge(cust_order_pay,pdt_item,how= 'left', left_on= 'order_id', right_on= 'order_id')

In [ ]:
byCat = cust_order_summ.groupby('product_category_name')[['payment_value','order_item_id']].sum().reset_index()
byCat.rename(columns= {'payment_value': 'Total_Sales','order_item_id':'Quantity'})

,product_category_name,Total_Sales,Quantity
0,Agro_Industry_And_Commerce,118730.61,316.0
1,Air_Conditioning,91170.66,368.0
2,Art,30992.93,240.0
3,Arts_And_Craftmanship,2326.17,26.0
4,Audio,60324.62,399.0
...,...,...,...
66,Stationery,317440.07,2915.0
67,Tablets_Printing_Image,10042.93,93.0
68,Telephony,486882.05,5349.0
69,Toys,619037.69,4611.0


In [ ]:
byLoc = cust_order_summ.groupby('customer_city')[['payment_value','order_item_id']].sum().reset_index()
byLoc.rename(columns= {'payment_value': 'Total_Sales','order_item_id':'Quantity'})

,customer_city,Total_Sales,Quantity
0,Aadityana,2244.73,33.0
1,Aambaliyasan,242.94,2.0
2,Aantaliya,16726.52,112.0
3,Aarambhada,7186.11,61.0
4,Abiramam,89.19,1.0
...,...,...,...
4114,Zamin Uthukuli,221.71,2.0
4115,Ziauddin Pur,25508.06,191.0
4116,Zira,568.43,5.0
4117,Zirakpur,600.64,4.0


In [ ]:
byMonth = cust_order_summ.groupby('order_purchase_month')[['payment_value','order_item_id']].sum().reset_index()
byMonth.rename(columns= {'payment_value': 'Total_Sales','order_item_id':'Quantity'})

,order_purchase_month,Total_Sales,Quantity
0,April,2.008735e+06,13301.0
1,August,2.124967e+06,14980.0
2,December,1.057602e+06,7648.0
3,February,1.660963e+06,12054.0
4,January,1.604364e+06,11532.0
5,July,2.104021e+06,14496.0
6,June,1.907577e+06,13190.0
7,March,2.024784e+06,13925.0
8,May,2.242130e+06,15055.0
9,November,1.599444e+06,11027.0


In [ ]:
byWeek = cust_order_summ.groupby(cust_order_summ.order_purchase_timestamp.dt.week)[['payment_value','order_item_id']].sum().reset_index()
byWeek.rename(columns= {'order_purchase_timestamp':'Week','payment_value': 'Total_Sales','order_item_id':'Quantity'})

<ipython-input-125-76d415083b0d>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  byWeek = cust_order_summ.groupby(cust_order_summ.order_purchase_timestamp.dt.week)[['payment_value','order_item_id']].sum().reset_index()


,Week,Total_Sales,Quantity
0,1,270023.950000,1963.0
1,2,388389.060000,2607.0
2,3,360990.270000,2701.0
3,4,389310.000000,2813.0
4,5,433286.320000,3057.0
5,6,399092.600000,2791.0
6,7,378639.150000,2900.0
7,8,427160.300000,3261.0
8,9,489233.520000,3328.0
9,10,472422.400000,3258.0


In [ ]:
byDay = cust_order_summ.groupby(cust_order_summ.order_purchase_timestamp.dt.day)[['payment_value','order_item_id']].sum().reset_index()
byDay.rename(columns= {'order_purchase_timestamp':'Day','payment_value': 'Total_Sales','order_item_id':'Quantity'})

,Day,Total_Sales,Quantity
0,1,689782.500000,4318.0
1,2,670149.030000,4485.0
2,3,650954.370000,4378.0
3,4,684087.500000,4592.0
4,5,740125.600000,4974.0
5,6,740143.890000,4820.0
6,7,700961.440000,4672.0
7,8,656822.650000,4750.0
8,9,672011.270000,4563.0
9,10,651772.380000,4373.0


In [ ]:
byTime = cust_order_summ.groupby(cust_order_summ.order_purchase_timestamp.dt.time)[['payment_value','order_item_id']].sum().reset_index()
byTime.rename(columns= {'order_purchase_timestamp':'Day','payment_value': 'Total_Sales','order_item_id':'Quantity'})

,Day,Total_Sales,Quantity
0,00:00:00,8725.39,60.0
1,00:01:00,8460.38,67.0
2,00:02:00,6824.77,54.0
3,00:03:00,10929.12,72.0
4,00:04:00,12188.89,95.0
...,...,...,...
1428,23:55:00,10435.77,80.0
1429,23:56:00,11495.29,88.0
1430,23:57:00,8897.06,57.0
1431,23:58:00,8591.81,54.0


In [ ]:
by_pay_method = cust_order_summ.groupby('payment_type')[['payment_value','order_item_id']].sum().reset_index()
by_pay_method.rename(columns= {'payment_value': 'Total_Sales','order_item_id':'Quantity'})

,payment_type,Total_Sales,Quantity
0,UPI,4.086821e+06,28640.0
1,credit_card,1.569535e+07,102947.0
2,debit_card,2.564179e+05,1953.0
3,not_defined,0.000000e+00,0.0
4,voucher,4.326022e+05,7105.0


## f. Popular Products by month, seller, state, category.

In [ ]:
cust_order_seller = pd.merge(cust_order_summ,sellers,how= 'left', left_on= 'seller_id', right_on= 'seller_id')

In [ ]:
cust_order_seller.groupby(['seller_id','order_purchase_month','seller_state','product_category_name'])[['order_item_id']].count().reset_index().sort_values(by='order_item_id', ascending= False).head().rename(columns={'order_item_id':'Qty','order_purchase_month':'Month','seller_state':'State'})

,seller_id,Month,State,product_category_name,Qty
2822,1f50f920176fa81dab994f9023523100,November,Andhra Pradesh,Garden_Tools,370
8540,6560211a19b47992c3666cc44a7e94c0,July,Andhra Pradesh,Watches_Gifts,264
8523,6560211a19b47992c3666cc44a7e94c0,August,Andhra Pradesh,Watches_Gifts,247
6219,4a3ca9315b744ce9f8e9374361493884,May,Gujarat,Bed_Bath_Table,223
10777,7c67e1448b00f6e969d365cea6b010ab,March,Andhra Pradesh,Office_Furniture,214


## g. Popular categories by state, month

In [ ]:
cust_order_seller.groupby(['order_purchase_month','seller_state','product_category_name'])[['order_item_id']].sum().reset_index().sort_values(by='order_item_id', ascending= False).head().rename(columns={'order_item_id':'Qty','order_purchase_month':'Month','seller_state':'State'})

,Month,State,product_category_name,Qty
1012,February,Andhra Pradesh,Computers_Accessories,1383.0
1670,July,Andhra Pradesh,Health_Beauty,1206.0
400,August,Andhra Pradesh,Health_Beauty,1067.0
2053,June,Andhra Pradesh,Health_Beauty,1063.0
2740,May,Andhra Pradesh,Health_Beauty,1019.0


## h. List top 10 most expensive products sorted by price

In [ ]:
pdt_item.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

In [ ]:
pdt_item.groupby('product_category_name')[['price']].sum().sort_values(by = 'price', ascending= False).head(10)

,price
product_category_name,
Health_Beauty,1258681.34
Watches_Gifts,1205005.68
Bed_Bath_Table,1036988.68
Sports_Leisure,988048.97
Computers_Accessories,911954.32
Furniture_Decor,729762.49
Cool_Stuff,635290.85
Housewares,632248.66
Auto,592720.11


# 2. Performing Customers/sellers Segmentation

## a. Divide the customers into groups based on the revenue generated

In [ ]:
p = cust_order_seller.payment_value
cust_order_seller['cust_grp'] = np.where(p <= 3416.02,'Q1',np.where(p <= 6832.04,'Q2',np.where(p <= 10248.06,'Q3','Q4')))
cust_order_seller[['customer_id','payment_value','cust_grp']]

,customer_id,payment_value,cust_grp
0,06b8999e2fba1a1fbc88172c00ba8bc7,146.87,Q1
1,18955e83d337fd6b2def6b18a428ac77,335.48,Q1
2,4e7b3e00288586ebd08712fdd0374a03,157.73,Q1
3,b2b6027bc5c5109e529d4dc6358b12c3,173.30,Q1
4,4f2d8ab171c80ec8364f7c12e35b23ad,252.25,Q1
...,...,...,...
118429,17ddf5dd5d51696bb3d7c6291687be6f,88.78,Q1
118430,e7b71a9017aa05c9a7fd292d714858e8,129.06,Q1
118431,5e28dfe12db7fb50a4b2f691faecea5e,56.04,Q1
118432,56b18e2166679b8a959d72dd06da27f9,711.07,Q1


## b. Divide the sellers into groups based on the revenue generated

In [ ]:
p = cust_order_seller.payment_value
cust_order_seller['seller_grp'] = np.where(p <= 3416.02,'S1',np.where(p <= 6832.04,'S2',np.where(p <= 10248.06,'S3','S4')))
cust_order_seller[['seller_id','payment_value','seller_grp']]

,seller_id,payment_value,seller_grp
0,7c67e1448b00f6e969d365cea6b010ab,146.87,S1
1,b8bc237ba3788b23da09c0f1f3a3288c,335.48,S1
2,7c67e1448b00f6e969d365cea6b010ab,157.73,S1
3,7c67e1448b00f6e969d365cea6b010ab,173.30,S1
4,4a3ca9315b744ce9f8e9374361493884,252.25,S1
...,...,...,...
118429,527801b552d0077ffd170872eb49683b,88.78,S1
118430,3fd1e727ba94cfe122d165e176ce7967,129.06,S1
118431,d9e7e7778b32987280a6f2cb9a39c57d,56.04,S1
118432,4869f7a5dfa277a7dca6462dcf3b52b2,711.07,S1


# 3. Cross-Selling (Which products are selling together)
Hint: We need to find which of the top 10 combinations of products are selling together in
each transaction. (combination of 2 or 3 buying together)

In [ ]:
t1 = cust_order_summ[['customer_id','order_id','product_category_name','order_item_id','payment_value']]
t2 = cust_order_summ[['customer_id','order_id','product_category_name','order_item_id','payment_value']]
t2.drop_duplicates(subset= 'product_category_name',keep= 'first', inplace= True)
t2 = cust_order_summ[['product_category_name']]
t2.drop_duplicates(keep= 'first', inplace= True)
t1['key'] = 1
t2['key'] = 1

In [ ]:
res = pd.merge(t1,t2,on='key').drop('key',1)
res[['product_category_name_x','product_category_name_y']]
res.groupby(['product_category_name_x','product_category_name_y']).agg(N=("order_id", 'count')).sort_values(by = 'N', ascending= False)

# 4. Payment Behaviour

## a. How customers are paying?

In [ ]:
print(cust_order_summ.payment_type.unique())

['credit_card' 'debit_card' 'voucher' 'UPI' 'not_defined']


## b. Which payment channels are used by most customers?

In [ ]:
cust_order_summ.groupby('payment_type')[['customer_id']].count().sort_values(by='customer_id', ascending= False).head(1).reset_index().rename(columns = {'customer_id':'Count_of_Customers'})

,payment_type,Count_of_Customers
0,credit_card,87289


# 5. Customer satisfaction towards category & product

## a. Which categories (top 10) are maximum rated & minimum rated?

In [ ]:
cust_order_rating = pd.merge(order_review_ratings, cust_order_summ, how='left', left_on = 'order_id', right_on = 'order_id')

In [ ]:
cust_order_rating.groupby('product_category_name')[['review_score']].sum().reset_index().sort_values(by = 'review_score', ascending= False).head(10)

,product_category_name,review_score
7,Bed_Bath_Table,46366
43,Health_Beauty,41318
65,Sports_Leisure,36856
39,Furniture_Decor,34407
15,Computers_Accessories,32002
49,Housewares,29854
70,Watches_Gifts,24856
68,Telephony,18612
42,Garden_Tools,18412
5,Auto,17770


In [ ]:
cust_order_rating.groupby('product_category_name')[['review_score']].sum().reset_index().sort_values(by = 'review_score').head(10)

,product_category_name,review_score
61,Security_And_Services,5
29,Fashion_Childrens_Clothes,36
52,La_Cuisine,63
11,Cds_Dvds_Musicals,65
3,Arts_And_Craftmanship,99
46,Home_Comfort_2,105
23,Diapers_And_Hygiene,127
32,Fashion_Sport,132
35,Flowers,139
41,Furniture_Mattress_And_Upholstery,156


## b. Which products (top10) are maximum rated & minimum rated?

In [ ]:
cust_order_rating.groupby('product_id')[['review_score']].sum().reset_index().sort_values(by = 'review_score', ascending= False).head(10)

,product_id,review_score
22112,aca2eb7d00ea1a7b8ebd4e68314663af,2149
19742,99a4788cb24856965c36a24e339b6058,2047
8613,422879e10f46682990de24d770e7f83d,1993
7364,389d119b48cf3043d311335e499d9c6b,1665
7079,368c6c730842d78016ad823897a372db,1553
10840,53759a2ecddad2bb87a079a1f1519f73,1516
27039,d1c427060a0f73f6b889a5c7c61f2ac4,1456
10867,53b36df67ebb7c41585e8d54d6772e08,1368
2794,154e7e31ebfa092203795c972e5804a6,1274
8051,3dd2a17168ec895c781a9191c1e95ad7,1163


In [ ]:
cust_order_rating.groupby('product_id')[['review_score']].sum().reset_index().sort_values(by = 'review_score').head(10)

,product_id,review_score
23527,b75683e29689c1a989ae97883e8cad56,1
26221,cb9ebc2581c01de678351c46f55f7f7f,1
2187,10a6596b3224c408ee46c8aaaa2977cd,1
29916,e87c931c62c20f4b62ea85af5e22538a,1
11302,56fd30e18420cb9a5a50c718168fa660,1
17406,871238c57d25f282d163be9e712c36a9,1
11300,56f96b94cbba67352e072d40e4e02167,1
17413,8722afd14143e0192b2abbcee6fe2873,1
17429,873c256958122a0b04f65c23f8bc2329,1
26246,cbd0f39c794fee28d71a874538c18f77,1


## c. Average rating by location, seller, product, category, month etc.

In [ ]:
cust_order_rating.groupby(['customer_city','seller_id','product_id','product_category_name','order_purchase_month'])[['review_score']].mean()

review_score
customer_city seller_id                        product_id                       product_category_name order_purchase_month              
Aadityana     2528513dd95219a6013d4d05176e391a ca0019454bd5ddca0f4022fe89f6557a Home_Confort          June                           5.0
              3b872fd4747f01cc56206f2934198618 77fdeb8d4f7502f75b6779205f30a875 Bed_Bath_Table        June                           5.0
              44073f8b7e41514de3b7815dd0237f4f 059dce566827cd3b3274f2b986d5c0db Garden_Tools          August                         1.0
              4a3ca9315b744ce9f8e9374361493884 84f456958365164420cfc80fbe4c7fab Bed_Bath_Table        October                        4.0
                                               c785fffd281ac9afffc38c8015455fd8 Bed_Bath_Table        March                          5.0
...                                                                                                                                  ...
Ziro          df91910b6a03bb2e3358fa6a35e32f6f 72ba1b1e37ca92467fe787728625c76a Housewares            June                           4.0
              e5a38146df062edaf55c38afa99e42dc 4ac50dbde931c0a5ed8c33d3dc047351 Health_Beauty         April                          4.0
              e6a69c4a27dfdd98ffe5aa757ad744bc 29ed7e120478b8a6ab3aeb253c6c7220 Housewares            May                            5.0
              e9779976487b77c6d4ac45f75ec7afe9 4f27f0b527bb700ed8541270356a38f3 Sports_Leisure        April                          5.0
              f457c46070d02cadd8a68551231220dd dbb67791e405873b259e4656bf971246 Computers_Accessories February                       1.0

[96639 rows x 1 columns]